In [5]:
import sys
sys.path.insert(0, '/Users/bairdhowland/Topic_Modelling')
from lda import LDA
import datetime
import csv
import pickle

In [2]:
def fix_nulls(s):
    for line in s:
        yield line.replace('\0', ' ')
        
def str_to_datetime(s):
    s = s.replace(' ','-').replace(':','-')
    return datetime.datetime(*[int(x) for x in s.split('-')])

def read_segments(f, min_duration=0, datetime_window=[]):
    segments = []
    csv_f = fix_nulls(open(f,'r'))
    for i,row in enumerate(csv.reader(csv_f)):
        if i==0: continue
        if int(row[6]) < min_duration:
            continue
        if datetime_window and datetime_window[0] < str_to_datetime(row[1]) <= datetime_window[1]:
            segments.append(row[7])
    csv_f.close()
    return segments



In [3]:
data_dir = "Transcript_Segments"
cnn_f = data_dir + '/CNN_full_segments.csv'
msnbc_f = data_dir + '/MSNBC_full_segments.csv'
fox_f = data_dir + '/FOXNEWS_full_segments.csv'

# Slice time into three-month intervals, learn model for each slice

In [7]:
dt = datetime.datetime(2016,11,22)
while dt < datetime.datetime(2019,1,1):
    window = [dt, dt + datetime.timedelta(days=90)]
    print(window)
    segments = (read_segments(cnn_f, min_duration=60, datetime_window=window) + 
                read_segments(fox_f, min_duration=60, datetime_window=window) + 
                read_segments(msnbc_f, min_duration=60, datetime_window=window))
    lda = LDA(segments)
    output_dir = "Topic-Modelling/models/" + str(dt).split()[0]
    for k in [10,20]:
        model = lda.train_lda(min_df=10, max_df=.6, n_topics=k, chunksize=50)
        lda.save_learned_topics(model, output_dir + '/topics_k={}.csv'.format(k), n_words=40)
        model.save(output_dir + '/model-gensim_k={}'.format(k))
    pickle.dump(lda.term_to_index, open(output_dir + '/term_to_index.p','wb'))
    pickle.dump(lda.index_to_term, open(output_dir + '/index_to_term.p','wb'))
    dt = dt + datetime.timedelta(days=90)

[datetime.datetime(2016, 11, 22, 0, 0), datetime.datetime(2017, 2, 20, 0, 0)]
Computing document-frequency dictionary; 3262 documents
100.0% completee
33726 total seen words
Training Model. 3262 documents. Vocabulary Size = 6150
Training Model. 3262 documents. Vocabulary Size = 6150
[datetime.datetime(2017, 2, 20, 0, 0), datetime.datetime(2017, 5, 21, 0, 0)]
Computing document-frequency dictionary; 3066 documents
100.0% completee
32632 total seen words
Training Model. 3066 documents. Vocabulary Size = 6008
Training Model. 3066 documents. Vocabulary Size = 6008
[datetime.datetime(2017, 5, 21, 0, 0), datetime.datetime(2017, 8, 19, 0, 0)]
Computing document-frequency dictionary; 3235 documents
100.0% completee
32060 total seen words
Training Model. 3235 documents. Vocabulary Size = 6177
Training Model. 3235 documents. Vocabulary Size = 6177
[datetime.datetime(2017, 8, 19, 0, 0), datetime.datetime(2017, 11, 17, 0, 0)]
Computing document-frequency dictionary; 3274 documents
100.0% completee

In [11]:
8*8

64